<a href="https://colab.research.google.com/github/nuot/AL-Clean/blob/main/AssistantListing_ObliClean_InProgress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Assistant Listing Obligation data cleaning project Version1** 
<br>

Author: Nuo Tian



**Last Updated: 03/16/2021**

Ask user input for the file name, target the obligation column

In [189]:
import pandas as pd
import re

filename = input("Enter name of input file (including the file extension):")
data = pd.read_csv(filename, encoding='cp1252')

Enter name of input file (including the file extension):AssistanceListings_DataGov_PUBLIC_WEEKLY_20210320.csv


Loop through column names, find the column name contains 'Obligation' and get the column name 

In [190]:
colnames = list(data.columns)
obligations_colname = [col for col in colnames if re.findall(r'Obligation', col)][0]
obligations = data[obligations_colname]

find all types in each row

In [191]:
types = []
for row in obligations:
  a = re.findall(r'\((.*?)\) FY',str(row))
  if a:
    types.append(a)
  else:
    types.append('None')

In [192]:
types_dedup = []
for type1 in types:
    type1 = list(dict.fromkeys(type1))
    types_dedup.append(type1)

In [193]:
data['Obligation Types'] = types_dedup

Next working on seprating the $amount for each obligation type

In [194]:
def listToStringWithoutBrackets(list1):
    return str(list1).replace('[','').replace(']','').replace("'","")

In [195]:
Obli_types = []
for i in range(len(data['Obligation Types'])):
  res = listToStringWithoutBrackets(data['Obligation Types'][i])
  Obli_types.append(res)

In [196]:
data['Obligation Types'] = Obli_types

In [197]:
maxlen = max(list(map(lambda x: len(x.split(",")) ,data['Obligation Types'])))
cols = ["Obligation"+str(x)   for x in range(maxlen)]
datalist = list(map(lambda x: x.split(","), data['Obligation Types']))
newdf = pd.DataFrame(data=datalist, columns=cols) 
data = pd.concat([data, newdf], axis=1) 

In [188]:
#mask = data.applymap(lambda x: x is None)
#cols = data.columns[(mask).any()]
#for col in data[cols]:
    #data.loc[mask[col], col] = ''

Amount

In [198]:
obli1 = obligations.str.split(')', expand=True)

In [199]:
df = pd.DataFrame()

i = 0
for i in range(len(obli1.columns)):
  val = []
  for row in obli1[i]:
    a = re.findall(r'FY [0-9][0-9]\$[^\]]+[.%]?',str(row))
    b = re.findall(r'FY [0-9][0-9] est \$[^\]]+[.%]?',str(row))
    if a:
      val.append(a)
    elif b:
      val.append(b)
    else:
      val.append('')
  df.insert(i, i, val, True) 


In [200]:
df = df.applymap(str)

In [201]:
import string

def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace("['", "")
        text = text.replace("- ']", "")
    return text

In [202]:
i = 0
while i < len(df.columns):
  df[i] = df[i].apply(remove_punctuations)
  i +=1

In [203]:
e1 = df[1][0].replace('$', ';')
e1 = e1.split(';')

In [169]:
for j in range(maxlen):
  FY1 = []
  FY2 = []
  FY2_Flag = []
  FY3 = []
  FY3_Flag = []
  name = 'Obligation'+str(j)
  pattern1 = e1[0] + '\$[^\]]+[.%]?'
  pattern2 = e1[2].lstrip() + '\$[^\]]+[.%]?'
  pattern2_1 = re.sub(' est', '', e1[2]).strip() + '\$[^\]]+[.%]?'
  pattern3 = e1[4].lstrip() + '\$[^\]]+[.%]?'
  pattern3_1 = re.sub(' est', '', e1[4]).strip() + '\$[^\]]+[.%]?'
  rep1 = e1[0]+'$'
  rep2 = re.sub(' est', '', e1[2]).strip() + '$'
  rep3 = e1[2].lstrip()+'$'
  rep4 = re.sub(' est', '', e1[4]).strip() + '$'
  rep5 = e1[4].lstrip()+'$'
  for i in range(len(df[j])):  
    a = re.findall(pattern1,df[j+1][i])
    b = re.findall(pattern2_1,df[j+1][i])
    c = re.findall(pattern2, df[j+1][i])
    d = re.findall(pattern3_1,df[j+1][i])
    e = re.findall(pattern3, df[j+1][i])
    if a :
      FY1.append(a[0].split(';')[0].replace(rep1,''))
    else:
      FY1.append('')
    if b:
      FY2.append(b[0].split(';')[0].replace(rep2,''))
      FY2_Flag.append(0)
    elif c:
      FY2.append(c[0].split(';')[0].replace(rep3,''))
      FY2_Flag.append(1)
    else:
      FY2.append('')
      FY2_Flag.append('')
    
    if d:
      FY3.append(d[0].split(';')[0].replace(rep4,''))
      FY3_Flag.append(0)
    elif e:
      FY3.append(e[0].split(';')[0].replace(rep5,''))
      FY3_Flag.append(1)
    else:
      FY3.append('')
      FY3_Flag.append('')

  data.insert(data.columns.get_loc(name) + 1, rep4.strip('$')+ '_Flag_' + str(j), FY3_Flag)
  data.insert(data.columns.get_loc(name) +1, rep4.strip('$') + '_' + str(j), FY3)
  data.insert(data.columns.get_loc(name) +1, rep2.strip('$') +'_Flag_' + str(j), FY2_Flag)
  data.insert(data.columns.get_loc(name) +1, rep2.strip('$')+ '_' + str(j), FY2)
  data.insert(data.columns.get_loc(name) +1, rep1.strip('$')+ '_' + str(j), FY1)

In [170]:
filename

'AssistanceListings_DataGov_PUBLIC_WEEKLY_20210213 - AssistanceListings_DataGov_PUBLIC_WEEKLY_20210213.csv'

In [171]:
reference_Data = re.findall('\d+', filename)[0]

In [172]:
data['Reference_Date'] = reference_Data

In [173]:
data.to_csv('AssistanceListingFinal.csv')